In [ ]:
import cv2
import os
import numpy as np
import torch
import numpy.fft as fft
import scipy.linalg as linalg
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib inline
from IPython.display import HTML, Video, display
from tqdm import tqdm

In [ ]:
%%sh
# Download the data - you need to do this only once
wget --no-verbose --output-document=image.jpg https://github.com/chrirupp/cv_course/raw/main/data/image.jpg
wget --no-verbose --output-document=image_ivy.jpg https://github.com/chrirupp/cv_course/raw/main/data/image_ivy.jpg

In [ ]:
class Visualizer():
    def __init__(self, num_rows=1, num_cols=1, figsize=(5,5), axis_off=True, title='', tight=False, cm=None):
        fig, self.axs = plt.subplots(num_rows, num_cols, figsize=figsize, squeeze=False)
        # remove ticks
        if axis_off:
          plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
        # set colormap
        if cm is not None:
            plt.set_cmap(cm)
        # set supertitle
        fig.suptitle(title)
        if tight:
            fig.subplots_adjust(top=0.88)

    def add_image_subplot(self, i, j, image, normalize=False, title_str=''):
        if normalize:
            image = self.normalize_image(image)
        if len(image.shape) == 3:
            #BGR -> RGB
            image = image[:, :, ::-1]
        self.axs[i, j].imshow(image)
        self.axs[i, j].set_title(title_str)

    def add_stem_subplot(self, i, j, x, y, title_str=''):
      self.axs[i, j].stem(x, y)
      self.axs[i, j].set_title(title_str)

    @staticmethod
    def normalize_image(image):
        img = np.float64(image) - np.min(image)
        img /= np.max(img)
        return img

def create_video(filename, fps, shape):
    return cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (shape[1], shape[0]))

def show_video(filename, width=640):
    tmp_mp4 = f"{filename[:-4]}_H264{filename[-4:]}"
    os.system(f"ffmpeg -i {filename} -vcodec libx264 {tmp_mp4}")
    os.remove(filename)
    os.rename(tmp_mp4, filename)
    display(Video(filename, embed=True, width=width))

In [ ]:
# Read image
image = cv2.imread('image.jpg')

vis = Visualizer(1, 1, figsize=(5,5))
vis.add_image_subplot(0, 0, image, title_str='Original image')


## Degradation examples

In [ ]:
# blur
blur_filter = cv2.getGaussianKernel(64, 6)
blur_filter = blur_filter @ blur_filter.T
blur_filter /= np.sum(blur_filter)
blur = cv2.filter2D(image, -1, blur_filter)

vis = Visualizer(1, 2, figsize=(15,5), axis_off=False)
vis.add_image_subplot(0, 0, blur_filter, title_str='Blur filter')
vis.add_image_subplot(0, 1, blur, title_str='Output image')


In [ ]:
# motion blur (not real motion blur)
kernel = np.zeros((64, 64))
kernel[32, :] = 1
kernel = kernel / 64
motion_blur = cv2.filter2D(image, -1, kernel)

vis = Visualizer(1, 2, figsize=(15,5), axis_off=False)
vis.add_image_subplot(0, 0, kernel, title_str='Motion blur filter')
vis.add_image_subplot(0, 1, motion_blur, title_str='Output image')


In [ ]:
# additive noise
noise = np.random.normal(0, 64, image.shape)
noise_image = np.clip(image + noise, 0, 255).astype(np.uint8)

vis = Visualizer(1, 2, figsize=(15,5), axis_off=False)
vis.add_image_subplot(0, 0, np.clip(noise+128, 0, 255).astype(np.uint8), title_str='Additive noise')
vis.add_image_subplot(0, 1, noise_image, title_str='Output image')


In [ ]:
# downsample (spatial quantization)
downsample = cv2.resize(image, (0,0), fx=0.125, fy=0.125)

vis = Visualizer(1, 2, figsize=(15,5), axis_off=False)
vis.add_image_subplot(0, 0, image, title_str='Original image')
vis.add_image_subplot(0, 1, downsample, title_str='Downsampled image')

## Restoration

In [ ]:
def get_magnitude(dft):
    dft_shift = np.fft.fftshift(dft, axes=(0,1))
    magnitude = np.clip(np.log(1+np.abs(dft_shift)), 0, None)
    magnitude -= np.min(magnitude)
    magnitude /= np.max(magnitude)
    magnitude *= 255
    return np.uint8(magnitude)


# inverting blur
blur_dft = fft.fft2(np.float32(blur), axes=(0,1))
blur_mag = get_magnitude(blur_dft)

pad_size = ((image.shape[0]-blur_filter.shape[0])//2, (image.shape[1]-blur_filter.shape[1])//2)
blur_filter_padded = np.pad(blur_filter, ((pad_size[0],pad_size[0]), (pad_size[1],pad_size[1])), 'constant')
blur_filter_dft = fft.fft2(np.float32(blur_filter_padded))
blur_filter_mag = get_magnitude(blur_filter_dft)

deblurred_dft = blur_dft / blur_filter_dft[:, :, np.newaxis]
deblurred_dft[deblurred_dft.shape[0]//2, :, :] = 0
deblurred_dft[:, deblurred_dft.shape[1]//2, :] = 0
filtered_mag = get_magnitude(deblurred_dft)
deblurred_image = np.real(fft.ifftshift(fft.ifft2(deblurred_dft, axes=(0,1)), axes=(0,1)))

vis = Visualizer(1, 5, figsize=(25,5), tight=True)
vis.add_image_subplot(0, 0, blur, title_str='Blurred image')
vis.add_image_subplot(0, 1, blur_mag, normalize=True, title_str='FT(Blurred image)')
vis.add_image_subplot(0, 2, blur_filter_mag, normalize=True, title_str='FT(Blur filter)')
vis.add_image_subplot(0, 3, filtered_mag, title_str='FT(Deblurred image)')
vis.add_image_subplot(0, 4, np.clip(deblurred_image, 0, 255).astype(np.uint8), title_str='Deblurred image')


In [ ]:
# inverting blur with a Wiener filter
wiener_filter = np.conj(blur_filter_dft) / (np.abs(blur_filter_dft)**2 + 1e-3)
wiener_filter_mag = get_magnitude(wiener_filter)

wiener_dft = blur_dft * wiener_filter[:, :, np.newaxis]
wiener_image = np.real(fft.ifftshift(fft.ifft2(wiener_dft, axes=(0,1)), axes=(0,1)))
filtered_mag = get_magnitude(wiener_dft)

vis = Visualizer(1, 5, figsize=(25,5), tight=True)
vis.add_image_subplot(0, 0, blur, title_str='Blurred image')
vis.add_image_subplot(0, 1, blur_mag, normalize=True, title_str='FT(Blurred image)')
vis.add_image_subplot(0, 2, wiener_filter_mag, normalize=True, title_str='FT(Wiener filter)')
vis.add_image_subplot(0, 3, filtered_mag, title_str='FT(Deblurred image)')
vis.add_image_subplot(0, 4, np.clip(wiener_image, 0, 255).astype(np.uint8), title_str='Deblurred image')


In [ ]:
# inverting blur with a Wiener filter - animation
output = create_video('deblur_animation.mp4', 30, image.shape)
for i in range(0, 60):
    sigma = 1 + 8 * (1+np.sin(i/60.0 * np.pi * 2))*0.5
    blur_filter = cv2.getGaussianKernel(64, sigma)
    blur_filter = blur_filter @ blur_filter.T
    blur_filter /= np.sum(blur_filter)
    pad_size = ((image.shape[0]-blur_filter.shape[0])//2, (image.shape[1]-blur_filter.shape[1])//2)
    blur_filter_padded = np.pad(blur_filter, ((pad_size[0],pad_size[0]), (pad_size[1],pad_size[1])), 'constant')
    blur_filter_dft = fft.fft2(np.float32(blur_filter_padded))

    wiener_filter = np.conj(blur_filter_dft) / (np.abs(blur_filter_dft)**2 + 1e-3)
    wiener_dft = blur_dft * wiener_filter[:, :, np.newaxis]
    wiener_image = np.real(fft.ifftshift(fft.ifft2(wiener_dft, axes=(0,1)), axes=(0,1)))
    output.write(np.clip(wiener_image, 0, 255).astype(np.uint8))
output.release()
show_video('deblur_animation.mp4')

In [ ]:
# inverting motion blur - try to find better parameters
est_angle = -3  # adjust the angle
est_motion_blur_length = 16  # adjust the length (must be multiple of 2)
est_noise_ratio = 0.03  # adjust the noise ratio

motion_blur = cv2.resize(cv2.imread('image_ivy.jpg'), (0,0), fx=0.25, fy=0.25)
rot_mat = cv2.getRotationMatrix2D((motion_blur.shape[1]//2, motion_blur.shape[0]//2), est_angle, 1.0)
motion_blur = cv2.warpAffine(motion_blur, rot_mat, (motion_blur.shape[1], motion_blur.shape[0]), flags=cv2.INTER_LINEAR)
motion_blur_dft = fft.fft2(np.float32(motion_blur), axes=(0,1))

motion_blur_filter = np.zeros((est_motion_blur_length, est_motion_blur_length))
motion_blur_filter[motion_blur_filter.shape[0]//2, :] = 1.0 / motion_blur_filter.shape[1]
pad_size = ((motion_blur.shape[0]-motion_blur_filter.shape[0])//2, (motion_blur.shape[1]-motion_blur_filter.shape[1])//2)
motion_blur_filter_padded = np.pad(motion_blur_filter, ((pad_size[0],pad_size[0]), (pad_size[1],pad_size[1])), 'constant')
motion_blur_filter_dft = fft.fft2(np.float32(motion_blur_filter_padded))
motion_blur_filter_mag = get_magnitude(motion_blur_filter_dft)

motion_wiener_filter = np.conj(motion_blur_filter_dft) / (np.abs(motion_blur_filter_dft)**2 + est_noise_ratio)
motion_wiener_filter_mag = get_magnitude(motion_wiener_filter)

motion_wiener_dft = motion_blur_dft * motion_wiener_filter[:, :, np.newaxis]
motion_wiener_image = np.real(fft.ifftshift(fft.ifft2(motion_wiener_dft, axes=(0,1)), axes=(0,1)))
rot_mat = cv2.getRotationMatrix2D((motion_wiener_image.shape[1]//2, motion_wiener_image.shape[0]//2), -est_angle, 1.0)
motion_wiener_image = cv2.warpAffine(motion_wiener_image, rot_mat, (motion_wiener_image.shape[1], motion_wiener_image.shape[0]), flags=cv2.INTER_LINEAR)
motion_filtered_mag = get_magnitude(motion_wiener_dft)

vis = Visualizer(1, 5, figsize=(25,5), tight=True)
vis.add_image_subplot(0, 0, motion_blur, title_str='Motion-blurred image')
vis.add_image_subplot(0, 1, motion_blur_filter_mag, normalize=True, title_str='FT(Motion-blur filter)')
vis.add_image_subplot(0, 2, motion_wiener_filter_mag, normalize=True, title_str='FT(Motion-blur Wiener filter)')
vis.add_image_subplot(0, 3, motion_filtered_mag, title_str='FT(Deblurred image)')
vis.add_image_subplot(0, 4, np.clip(motion_wiener_image, 0, 255).astype(np.uint8), title_str='Deblurred image')

In [ ]:
# inverse problems
# create an mirror overlay image
noise = np.random.normal(0, 64, image.shape)
ratio = 0.4
image_mirror = np.uint8(np.clip((ratio*np.float32(image) + (1-ratio)*np.float32(image[:, ::-1, :])) + noise, 0, 255))
# cv2.imwrite('04/image_mirror.jpg', image_mirror)
vis = Visualizer(1, 1, figsize=(5, 5))
vis.add_image_subplot(0, 0, image_mirror, title_str=f'Corrupted image (ratio:{ratio} mirrored)')
plt.show()

print('Restoring original image (demirror)')
low_res = cv2.resize(image_mirror, (0,0), fx=0.5, fy=0.5)  # consider downsampling more - it is quite slow
output = create_video('demirror_animation.mp4', 30, low_res.shape)
target = torch.from_numpy(low_res).float()
image_est = torch.nn.Parameter(torch.zeros(low_res.shape, dtype=torch.float32))
for i in tqdm(range(300*30)):
    reconstruction = ratio*image_est + (1-ratio)*torch.fliplr(image_est)
    reconstruction_error = torch.mean((reconstruction - target)**2)
    gradient_x = image_est[:, 1:, :] - image_est[:, :-1, :]
    gradient_y = image_est[1:, :, :] - image_est[:-1, :, :]
    prior = torch.mean(torch.abs(gradient_x)) + torch.mean(torch.abs(gradient_y))
    loss = reconstruction_error + 2 * prior
    # gradient descent:
    image_est.grad = None  # clear gradients
    loss.backward()  # compute gradients
    image_est.data -= 1000 * image_est.grad.data  # do a gradient step
    if i % 30 == 0:
        output.write(np.clip(image_est.data.numpy(), 0, 255).astype(np.uint8))
output.release()
show_video('demirror_animation.mp4')